# Adversarial samples generator

In [ ]:
import torch
import torchvision
import numpy as np
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import time

from torchattacks import FGSM, CW, FAB, OnePixel, DeepFool

from nonMathAttacks import NonMathAttacks

from imageLimitedDataset import ImageLimitedDataset

print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

In [ ]:
# Detect if we have a GPU available
print("CUDA available:", torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
nonMathAttacks = NonMathAttacks()

In [ ]:
adversarialDir = "./adversarial_samples"
datasetsDir = "./datasets"
modelsDir = "./models"

## Utils

In [ ]:
def getSubDirs(dir):
    return [x for x in os.listdir(dir) if os.path.isdir(os.path.join(dir, x))]

In [ ]:
def getClassPercents(sizes):
    totalSize = np.sum(np.array(sizes))
    percents = []
    for size in sizes:
        percents.append(int(round((size / totalSize) * 100)))

    return percents


In [ ]:
def saveMathAdversarials(dataloader, classes, fileNames, attack, saveDir):

    i = 0;
    for images, labels in dataloader:
        adversarials = attack(images, labels)

        for adversarial, label in zip(adversarials, labels):
            image = transforms.ToPILImage()(adversarial).convert("RGB")
            path = os.path.join(saveDir, classes[label])

            if not os.path.exists(path):
                os.makedirs(path)

            imageName = os.path.basename(fileNames[i][0])
            image.save(os.path.join(path, imageName), "JPEG")

            i += 1

## Generate adversarials

In [ ]:
attacks = {
    "GaussianNoise": nonMathAttacks.gaussianNoise,
    "BoxBlur": nonMathAttacks.boxBlur,
    "Sharpen": nonMathAttacks.sharpen,
    "InvertColor": nonMathAttacks.invertColor,
    "GreyScale": nonMathAttacks.greyscale,
    "SplitMergeRGB": nonMathAttacks.splitMergeRGB,
    "RandomBlackBox": nonMathAttacks.randomBlackBox,
    "SaltPepper": nonMathAttacks.saltAndPepper,
}

In [ ]:
timesEvaluations = []

for dataset in getSubDirs(datasetsDir):
    print("\n" + "-" * 15)
    currentTime = time.time()
    print("[🗃️ TEST DATASET] {}".format(dataset))

    datasetDir = os.path.join(datasetsDir, dataset)
    testDir = os.path.join(datasetDir, "test")

    datasetAdvDir = os.path.join(adversarialDir, dataset)
    nonMathAttacksDir = os.path.join(datasetAdvDir, "nonMath")

    if not os.path.exists(nonMathAttacksDir):
        os.makedirs(nonMathAttacksDir)

    testDataset = ImageLimitedDataset(testDir, use_cache=True, check_images=False)

    for path, cls in testDataset.imgs:
        clsName = testDataset.classes[cls]

        imageName = os.path.basename(path)

        image = Image.open(path).convert("RGB")

        for attack in attacks:
            attacker = attacks[attack]
            
            attackDir = os.path.join(nonMathAttacksDir, attack)
            saveDir = os.path.join(attackDir, clsName)

            if not os.path.exists(saveDir):
                os.makedirs(saveDir)

            outImage = attacker(image)
            outImage.save(os.path.join(saveDir, imageName), "JPEG")
    
    timesEvaluations.append({
        "dataset": dataset,
        "type": "nonMath",
        "attack": None,
        "model": None,
        "modelDataset": None,
        "balancing": None,
        "time": time.time() - currentTime,
    })

In [ ]:
for dataset in getSubDirs(datasetsDir):
    print("\n" + "-" * 15)
    print("[🗃️ TEST DATASET] {}".format(dataset))

    datasetDir = os.path.join(datasetsDir, dataset)
    testDir = os.path.join(datasetDir, "test")

    datasetAdvDir = os.path.join(adversarialDir, dataset)
    mathAttacksDir = os.path.join(datasetAdvDir, "math")

    if not os.path.exists(mathAttacksDir):
        os.makedirs(mathAttacksDir)

    toTensor = transforms.Compose([transforms.ToTensor()])
    testDataset = ImageLimitedDataset(
        testDir, transform=toTensor, use_cache=False, check_images=False)

    testDataLoader = DataLoader(
        testDataset, batch_size=16, num_workers=0)

    for root, _, fnames in sorted(os.walk(os.path.join(modelsDir, dataset), followlinks=True)):
        for fname in sorted(fnames):
            path = os.path.join(root, fname)
            modelData = torch.load(path)

            modelDataset = modelData["dataset"]
            modelName = modelData["model_name"]
            modelPercents = "_".join([str(x)
                                      for x in getClassPercents(modelData["dataset_sizes"])])
            model = modelData["model"].to(device)

            attacks = {
                "FGSM": FGSM(model),
                "CW": CW(model),
                "FAB": FAB(model),
                "OnePixel": OnePixel(model),
                "DeepFool": DeepFool(model),
            }

            for attack in attacks:
                attacker = attacks[attack]

                attackDir = os.path.join(
                    mathAttacksDir, attack)
                saveDir = os.path.join(
                    attackDir, modelName + "/" + modelPercents)
                
                if not os.path.exists(saveDir):
                    os.makedirs(saveDir)

                currentTime = time.time()
                print("[⚔️ ADVERSARIAL] {} - {} - {} {}".format(
                    attack,
                    modelDataset,
                    modelName,
                    modelPercents
                ))

                saveMathAdversarials(testDataLoader, testDataset.classes, testDataset.imgs, attacker, saveDir)

                timesEvaluations.append({
                    "dataset": dataset,
                    "type": "math",
                    "attack": attack,
                    "model": modelName,
                    "modelDataset": modelDataset,
                    "balancing": modelPercents.replace("_", "/"),
                    "time": time.time() - currentTime,
                })

                torch.cuda.empty_cache()

In [ ]:
import pandas as pd

modelsEvalsDF = pd.DataFrame(timesEvaluations)


In [ ]:
timesEvaluations